<a href="https://colab.research.google.com/github/Gustavo-RibMartins/estudos-python/blob/develop/curso/python_15_decoradores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Funções de maior grandeza

Higher Order Functions - HOF

Quando uma linguagem de programação suporta HOF, indica que podemos ter funções que retornam outras funções como resultado, ou mesmo que podemos passar funções como argumentos para outras funções, e até mesmo criar variáveis do tipo de funções nos nossos programas.

In [ ]:
def somar(a, b):
  return a + b

def diminuir(a, b):
  return a - b

def multiplicar(a, b):
  return a * b

def dividir(a, b):
  return a / b

def calcular(num1, num2, funcao):
  return funcao(num1, num2)

print(calcular(2, 3, somar)) # soma
print(calcular(2, 1, diminuir)) # subtracao

5
1


Em Python, as funções são **Cidadãos de Primeira Classe** (First Class Citizen).

---

**Nested Functions - Funções Aninhadas**

Em Python, também podemos ter funções dentro de funções, que são conhecidas por *Nested Functions*, ou também *Inner Functions*.

In [ ]:
# exemplo

from random import choice

def cumprimento(pessoa):
  def humor():
    return choice(('E ai, ', 'Suma daqui, ', 'Gosto muito de você, '))
  return humor() + pessoa

cumprimento('Gustavo')

'E ai, Gustavo'

In [ ]:
# retornando funções de outras funções

from random import choice

def faz_me_rir():
  def rir():
    return choice(('hahahhahaha', 'kkkkkkk', 'rsrsrsrs'))
  return rir

rindo = faz_me_rir()
print(rindo())
print(type(rindo))

rsrsrsrs
<class 'function'>


In [ ]:
from random import choice

def faz_me_rir(pessoa):
  def dando_risada():
    risada = choice(('hahahhahaha', 'kkkkkkk', 'rsrsrsrs'))
    return f'{risada} {pessoa}' # "pessoa" vem da função anterior
  return dando_risada

rindo = faz_me_rir('Gustavo')
print(rindo())
print(rindo())
print(rindo())

kkkkkkk Gustavo
hahahhahaha Gustavo
kkkkkkk Gustavo


Inner Functions podem acessar o escopo de funções mais externas.
Por exemplo, a função `rir()` consegue acessar o escopo de `faz_me_rir()`.

# 2.Decoradores

Decorators: são funções. Envolvem outras funções e aprimoram seus comportamentos.

Decorators também são exemplos de Higher Order Functions e possuem uma sintaxe própria usando `@` (Syntact Sugar/ Açúcar Sintático).

In [ ]:
# Decorators como funções (sintaxe não recomendada, sem @)

def seja_educado(funcao): # decorator
  def sendo():
    print('Foi um prazer conhecer você!')
    funcao()
    print('Tenha um ótimo dia!')
  return sendo

def saudacao(): # função decorada
  print('Seja bem-vindo(a)!')

# Testando

teste = seja_educado(saudacao)
teste()

Foi um prazer conhecer você!
Seja bem-vindo(a)!
Tenha um ótimo dia!


In [ ]:
# Decoratos como funções - Syntax Sugar (recomendado)

def seja_educado_mesmo(funcao):
  def sendo_mesmo():
    print('Foi um prazer conhecer você!')
    funcao()
    print('Tenha um excelente dia!')
  return sendo_mesmo

@seja_educado_mesmo
def apresentando():
  print('Meu nome é Gustavo')

# teste

apresentando()

Foi um prazer conhecer você!
Meu nome é Gustavo
Tenha um excelente dia!


# 3.Decoradores com diferentes assinaturas

Padrão de projeto: Decorator Pattern. Para resolver quando passamos mais parâmetros do que a função espera receber.

In [ ]:
def gritar(funcao):
  def aumentar(*args, **kwargs):
    return funcao(*args, **kwargs).upper()
  return aumentar

@gritar
def saudacao(nome):
  return f'Olá, eu sou o Gustavo'

@gritar
def ordenar(principal, acompanhamento):
  return f'Olá, eu gostaria de {principal} acompanhado de {acompanhamento}, por favor.'

print(saudacao('Gustavo'))
print(ordenar('Picanha', 'Batata Frita'))

OLÁ, EU SOU O GUSTAVO
OLÁ, EU GOSTARIA DE PICANHA ACOMPANHADO DE BATATA FRITA, POR FAVOR.


Assinatura de uma função é representada pelo seu retorno, nome e parâmetros de entrada. Em Python não temos retorno, então é apenas o nome da funação e seus parâmetros.

In [ ]:
# Decorator com argumentos

def verifica_primeiro_argumento(valor):
  def interna(funcao):
    def outra(*args, **kwargs):
      if args and args[0] != valor:
        return f'Valor incorreto! Primeiro argumento precisa ser {valor}'
      return funcao(*args, **kwargs)
    return outra
  return interna

@verifica_primeiro_argumento('pizza')
def comida_favorita(*args):
  print(args)

@verifica_primeiro_argumento(10)
def soma_dez(num1, num2):
  return num1 + num2

# testando

print(soma_dez(10, 20))
print(soma_dez(5, 20))

print(comida_favorita('pizza', 'churrasco', 'bolo', 'sorvete'))
print(comida_favorita('churrasco', 'bolo', 'sorvete'))

30
Valor incorreto! Primeiro argumento precisa ser 10
('pizza', 'churrasco', 'bolo', 'sorvete')
None
Valor incorreto! Primeiro argumento precisa ser pizza


# 4.Preservando Metadatas com wraps

Metadatas -> são dados intrínsecos em arquivos (tamanho, data de modificação,...)

wraps -> são funções que envolvem elementos com diversas finalidades.

In [3]:
# Problema

def ver_log(funcao):
  def logar(*args, **kwargs):
    """Eu sou uma função (logar) dentro de outra"""
    print(f'Você está chamando {funcao.__name__}')
    print(f'Aqui a documentação: {funcao.__doc__}')
    return funcao(*args, **kwargs)
  return logar

@ver_log
def soma(a, b):
  """Soma dois números"""
  return a + b

print(soma(10, 30)) # até aqui tudo ok

print(soma.__name__) # não trouxe o __name__ de soma()
print(soma.__doc__) # não trouxe a __doc__ de soma()

40
soma
Soma dois números


Para resolver isso, precisamos fazer import de um decorator da própria linguagem Python.

In [4]:
from functools import wraps

def ver_log(funcao):
  @wraps(funcao) # só isso
  def logar(*args, **kwargs):
    """Eu sou uma função (logar) dentro de outra"""
    print(f'Você está chamando {funcao.__name__}')
    print(f'Aqui a documentação: {funcao.__doc__}')
    return funcao(*args, **kwargs)
  return logar

@ver_log
def soma(a, b):
  """Soma dois números"""
  return a + b

print(soma(10, 30))

print(soma.__name__)
print(soma.__doc__)

Você está chamando soma
Aqui a documentação: Soma dois números
40
soma
Soma dois números


# 5.Forçando tipos de dados com um decorador

In [10]:
def forca_tipo(*tipos):
  def decorador(funcao):
    def converte(*args, **kwargs):
      novo_args = []
      for(valor, tipo) in zip(args, tipos):
        novo_args.append(tipo(valor))
      return funcao(*novo_args, **kwargs)
    return converte
  return decorador

@forca_tipo(str, int) # força tipos str e int
def repete_msg(msg, vezes):
  for vez in range(vezes):
    print(msg)

repete_msg('Guga', '3')

@forca_tipo(float, float)
def dividir(a, b):
  print(a/b)

dividir('1', '5')

Guga
Guga
Guga
0.2
